In [40]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
#from sklearn import cluster
from kneed import KneeLocator

In [121]:
#read the pkl file for movies
df = pd.read_pickle('./ABC/full_movies.pkl')
#read the pkl file for shows
df_2 = pd.read_pickle('./ABC/full_shows.pkl')

In [122]:
df.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
11,11,Family,Ballet Now,NaN,Ballet Now,This film captures the creative process behind...,This film captures the creative process behind...,"[ABC TV Plus, DOCUMENTARY, ARTS & CULTURE, FAM...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW269...,[],"[abc2, docs, factual, arts, ballet, dance, cul...",2021-06-30 21:59:27,M,3555.0,NaN,,Ballet Now
35,35,Family,Zog,NaN,Zog,"Based on the picture book by Julia Donaldson, ...","Based on the picture book by Julia Donaldson, ...","[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW124...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4feature, 4featured, abcme-shoulder...",2020-04-17 11:45:30,G,1621.0,NaN,,Zog
111,111,Family,Armadillo: Narrated By David Attenborough,NaN,Armadillo: Narrated By David Attenborough,In the heart of Brazil lives an animal so elus...,In the heart of Brazil lives an animal so elus...,"[ABC TV, DOCUMENTARY, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW311...,[Host Sir David Attenborough],"[abc1, uk, docs, factual, nature, wildlife, en...",2021-09-19 19:41:30,PG,3134.0,NaN,,Armadillo: Narrated By David Attenborough
137,137,Family,Stick Man,NaN,Stick Man,Stick Man lives in the family tree with his St...,Stick Man lives in the family tree with his St...,"[ABC Kids, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW002...,"[Director Jeroen Jaspaert, Daniel Snaddon]","[abc4kids, family-viewing, 4xmas-features, abc...",2020-04-17 11:44:30,G,1626.0,NaN,,Stick Man
158,158,Family,The Snail And The Whale,NaN,The Snail And The Whale,"A beautiful tale of an unlikely friendship, a ...","A beautiful tale of an unlikely friendship, a ...","[ABC Kids, ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW177...,"[Director Max Lang, Daniel Snaddon]","[abc4kids, 4featured, 4feature, 4family, 4uppe...",2020-04-17 11:46:30,G,1601.0,NaN,,The Snail And The Whale


In [123]:
df_2.head()

,category,description,description2,directors_actors,publication_date,rating,duration_sec,tags,tags2
title,,,,,,,,,
199 Little Heroes,Education,The journey to school is a very special type o...,"[Enjo lives in Quinten, a forest glade in the ...",[],2021-06-01 06:20:00,G,"[311.0, 311.0, 312.0, 311.0, 311.0, 320.0, 311...","[ABC ME, EDUCATION]","[environment-day, refugee-day, education, prim..."
7.30,News,Leigh Sales presents Australia's leading night...,[The devastation caused by the recent floods i...,[],2022-03-14 20:00:00,None,"[1906.0, 1894.0, 1879.0, 1935.0, 1891.0, 1822....","[ABC TV, ABC NEWS]","[pandemic, analysis, budget, flooding, vaccine..."
7.30 Mark Humphries Satire,News,Satirist Mark Humphries brings his unique pers...,[Satirist Mark Humphries goes inside the Healt...,[],2021-05-11 14:00:00,None,"[145.0, 110.0, 118.0, 136.0, 136.0, 202.0, 115...",[ABC NEWS],"[parachuting, news, news24, duty, candidates, ..."
7.30: The Interviews,News,Revisit some of 7.30's most compelling convers...,[Musician Dave Grohl sits down with 7.30's Lei...,[],2022-03-08 20:00:00,None,"[1030.0, 1030.0]",[ABC NEWS],"[news, news24]"
72 Cutest Animals,Family,The animal kingdom continually astounds us wit...,[Cute is a notion that can mean many things to...,[],2022-03-05 15:03:06,G,"[1616.0, 1616.0, 1620.0, 1620.0, 1614.0, 1620....","[ABC ME, FAMILY]","[family-viewing-abcme, abc3, family-viewing]"


In [43]:
#read json file activity for movies
activity = pd.read_json('./activities.json')

In [44]:
activity.head()
activity.groupby('user_id').count()

,content_id,activity,datetime
user_id,,,
1,100,100,100
2,108,108,108
3,101,101,101
4,83,83,83
5,108,108,108


## TF-IDF + kmeans clustering

In [45]:
#tf-idf and kmeans for all movies
# Convert the text data to a matrix of TF-IDF features
tfidf_vectorizer = TfidfVectorizer(stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(df['description'])


inertia = []
#calculate the optimal number of clusters
K = range(1, 20)
for k in K:
    kmeanModel = KMeans(n_clusters=k,random_state=0)
    kmeanModel.fit(tfidf)
    inertia.append(kmeanModel.inertia_)


kn = KneeLocator(K, inertia, curve='convex', direction='decreasing')
print(kn.knee)


13


In [85]:
# Cluster the documents using KMeans
num_clusters = 13
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf)

# Assign cluster labels to each document
clusters = km.labels_.tolist()
df['cluster'] = clusters

In [86]:
expat_library = df[df['tags2'].str.contains('Australia|australia|australian|Australian') | df['description'].str.contains('Australia|australia|australian|Australian') | df['description2'].str.contains('Australia|australia|australian|Australian') | df['title'].str.contains('Australia|australia|australian|Australian')
]
expat_library.head()

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,cluster
237,237,Family,Romeo & Juliet,NaN,Romeo & Juliet,"Graeme Murphy's Romeo and Juliet, performed by...","Graeme Murphy's Romeo and Juliet, performed by...","[ABC TV Plus, ABC ARTS, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW230...,[],"[abc2, arts, abcarts, ballet, entertainment, c...",2021-06-18 07:00:00,PG,7168.0,NaN,,Romeo & Juliet,7
404,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,11
424,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!,12
484,484,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End,3
595,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman,6


In [68]:
# Print the top 10 terms for each cluster
order_centroids = km.cluster_centers_.argsort()[:, ::-1]
terms = tfidf_vectorizer.get_feature_names()
for i in range(num_clusters):
    print("Cluster %d:" % i, end='')
    for ind in order_centroids[i, :13]:
        print(' %s' % terms[ind], end='')
    print()



Cluster 0: attenborough david making sir series scenes natural monsters look documentary flying museum galapagos
Cluster 1: school work high church paquita special pearl zog gadabout king iii biggest force
Cluster 2: beach stranded summer day lost time recalls gave flashback cadets neville kangaroo deserted
Cluster 3: book world based julia friendship donaldson tale beautiful tour scheffler axel mouse children
Cluster 4: process artist human family political personal chinese world creative stop scenes freedom iconic
Cluster 5: performed ballet orchestra australian music love victoria opera feisty costumes symphony australia production
Cluster 6: disability science people international greatest story geography epic live natural attempt save scientists
Cluster 7: gospel stunning music talented mark gorton lisser monastery semi manchester week final adventure
Cluster 8: lives children look young fielding home protect far reality 2017 film middle stage
Cluster 9: documentary intimate austr

/Users/ekaterinamazur/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [98]:
joined_activity = pd.merge(df, activity, left_on='id', right_on='content_id', how='right')
#make from list in tag2 column to string and delete [] in the string
joined_activity['tags2'] = joined_activity['tags2'].apply(lambda x: str(x).replace('[', '').replace(']', ''))
#drop NaN values in id column
joined_activity.dropna(subset=['id'], inplace=True)
#drop columns that are not needed
joined_activity.drop(['series', 'episode_name', 'description2', 'publication_date', 'rating', 'season', 'episode', 'episode_title', 'datetime', 'duration_sec', 'tags'], axis=1, inplace=True)
joined_activity.head()

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
1,366.0,Arts,Blasko,"A pioneer of indie music, Sarah Blasko walks t...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC154...,[Narrator Sarah Blasko],"'abc2', 'arts', 'docs', 'australia', 'music', ...",3.0,366,Like,3
5,379.0,Drama,The Boys Are Back,"Journalist Joe Warr and his two sons, from dif...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW238...,[Director Scott Hicks],"'abc1', 'abc2', 'drama', 'comedy-drama', 'film...",10.0,379,Play,2
6,298.0,Documentary,Joanna Lumley And The Human Swan,This inspiring film sees Joanna Lumley follow ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW324...,[Host Joanna Lumley],"'abc1', 'uk', 'factual', 'docs', 'travel', 'ad...",3.0,298,Dislike,1
9,451.0,Drama,Lore,"As the Allies sweep across Germany, Lore leads...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW277...,[Director Cate Shortland],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",3.0,451,Like,3
10,286.0,Documentary,Jaimen Hudson: From Sky To Sea,"Challenging stereotypes, this inspiring film c...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW316...,[],"'abc2', 'australia', 'arts', 'docs', 'factual'...",8.0,286,Dislike,4


In [70]:
#select rows where activity is 'Like'
expat_data_likes = joined_activity[joined_activity['activity'] == 'Like']
expat_data_likes

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
1,366.0,Arts,Blasko,"A pioneer of indie music, Sarah Blasko walks t...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC154...,[Narrator Sarah Blasko],"'abc2', 'arts', 'docs', 'australia', 'music', ...",9.0,366,Like,3
9,451.0,Drama,Lore,"As the Allies sweep across Germany, Lore leads...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW277...,[Director Cate Shortland],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",9.0,451,Like,3
11,366.0,Arts,Blasko,"A pioneer of indie music, Sarah Blasko walks t...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC154...,[Narrator Sarah Blasko],"'abc2', 'arts', 'docs', 'australia', 'music', ...",9.0,366,Like,5
35,47.0,Drama,Adoration,Two lifelong best friends each begin a steamy ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW237...,[Director Anne Fontaine],"'abc1', 'abc2', 'drama', 'sydney', 'romance', ...",12.0,47,Like,2
40,456.0,Drama,Being Julia,Successful stage actress Julia finds herself s...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Istvan Szabo],"'abc1', 'abc2', 'drama', 'comedy-drama', 'clas...",12.0,456,Like,1
41,456.0,Arts,Pinchgut Opera: Handel in Italy,"Erin Helyard, Taryn Fiebig and the musicians o...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW247...,[],"'abcarts', 'arts', 'music', 'classical', 'perf...",3.0,456,Like,1
42,399.0,Documentary,The Making Of David Attenborough's Kingdom Of ...,Take a glimpse behind the scenes at the tricks...,https://cdn.iview.abc.net.au/thumbs/i/zx/ZX915...,[Host Sir David Attenborough],"'abc2', 'docs', 'nature', 'uk', 'featured'",0.0,399,Like,4
43,399.0,Drama,On A Clear Day,Frank determines to salvage his self-esteem an...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW291...,[Director Gaby Dellal],"'abc1', 'abc2', 'uk', 'drama', '2000s', 'famil...",12.0,399,Like,4
62,83.0,Drama,Newcastle,Combining sublime surfing beaches with the raw...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW290...,[Director Dan Castle],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",12.0,83,Like,3
83,43.0,Drama,Adaptation,A lovelorn screenwriter struggles to adapt 'Th...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Spike Jonze],"'abc1', 'abc2', 'drama', 'comedy-drama', 'surr...",12.0,43,Like,2


In [115]:
#top-3 clusters for each user in expat_data_likes
top3 = expat_data_likes.groupby('user_id')['cluster'].value_counts().groupby(level=0).nlargest(3).reset_index(level=1, drop=True)
#make from series to dataframe
top3 = top3.to_frame()
#drop the 3rd column
top3 = top3.drop(columns=['cluster'])
#make user_id column as index
top3 = top3.reset_index()
top3

,user_id,cluster
0,1,1.0
1,1,12.0
2,1,5.0
3,2,12.0
4,2,4.0
5,2,3.0
6,3,9.0
7,3,12.0
8,3,3.0
9,4,9.0


In [81]:
#filter expat_data_likes dataframe by another dataframe with two columns: user_id and cluster
#this is only activity 'Like' for top-3 clusters for each user
expat_data_top = expat_data_likes[expat_data_likes[['user_id', 'cluster']].apply(tuple, axis=1).isin(top3.index)]
expat_data_top

,id,category,title,description,image,directors_actors,tags2,cluster,content_id,activity,user_id
1,366.0,Arts,Blasko,"A pioneer of indie music, Sarah Blasko walks t...",https://cdn.iview.abc.net.au/thumbs/i/ac/AC154...,[Narrator Sarah Blasko],"'abc2', 'arts', 'docs', 'australia', 'music', ...",9.0,366,Like,3
9,451.0,Drama,Lore,"As the Allies sweep across Germany, Lore leads...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW277...,[Director Cate Shortland],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",9.0,451,Like,3
35,47.0,Drama,Adoration,Two lifelong best friends each begin a steamy ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW237...,[Director Anne Fontaine],"'abc1', 'abc2', 'drama', 'sydney', 'romance', ...",12.0,47,Like,2
40,456.0,Drama,Being Julia,Successful stage actress Julia finds herself s...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Istvan Szabo],"'abc1', 'abc2', 'drama', 'comedy-drama', 'clas...",12.0,456,Like,1
42,399.0,Documentary,The Making Of David Attenborough's Kingdom Of ...,Take a glimpse behind the scenes at the tricks...,https://cdn.iview.abc.net.au/thumbs/i/zx/ZX915...,[Host Sir David Attenborough],"'abc2', 'docs', 'nature', 'uk', 'featured'",0.0,399,Like,4
62,83.0,Drama,Newcastle,Combining sublime surfing beaches with the raw...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW290...,[Director Dan Castle],"'abc1', 'abc2', 'drama', 'film', 'feature-leng...",12.0,83,Like,3
83,43.0,Drama,Adaptation,A lovelorn screenwriter struggles to adapt 'Th...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW300...,[Director Spike Jonze],"'abc1', 'abc2', 'drama', 'comedy-drama', 'surr...",12.0,43,Like,2
139,326.0,Kids,Stick Man,Stick Man lives in the family tree with his St...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW002...,"[Director Jeroen Jaspaert, Daniel Snaddon]","'abc4kids', 'family-viewing', '4xmas-features'...",12.0,326,Like,5
143,305.0,Drama,Goldstone,Indigenous detective Jay Swan arrives in the t...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW109...,[Director Ivan Sen],"'abc1', 'abc2', 'drama', 'film', 'detective', ...",12.0,305,Like,1
146,47.0,Drama,Adoration,Two lifelong best friends each begin a steamy ...,https://cdn.iview.abc.net.au/thumbs/i/zw/ZW237...,[Director Anne Fontaine],"'abc1', 'abc2', 'drama', 'sydney', 'romance', ...",12.0,47,Like,1


In [96]:
#check what from expat_library is not in expat_data_top by content_id
#that's ti check that we don't recommend movies that user already watched
suggestion = expat_library[~expat_library['id'].isin(expat_data_top['id'])]
suggestion

,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title,cluster
404,404,Family,Can We Save The Reef?,NaN,Can We Save The Reef?,An epic story of Australian and international ...,An epic story of Australian and international ...,"[ABC TV, DOCUMENTARY, SCIENCE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/sc/SC160...,[],"[abc1, docs, nature, science, science-week, cl...",2018-05-12 14:58:55,G,3428.0,NaN,,Can We Save The Reef?,11
424,424,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!,12
484,484,Family,Australia: The Wild Top End,NaN,Australia: The Wild Top End,Come with us on an epic journey through some o...,Come with us on an epic journey through some o...,"[ABC ME, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW284...,[],"[abc3, 3featured, family-viewing-abcme, family...",2022-03-16 09:45:00,PG,2602.0,NaN,,Australia: The Wild Top End,3
595,595,Family,Freeman,NaN,Freeman,The story of a nation coming together around I...,The story of a nation coming together around I...,"[ABC TV, ABC ME, INDIGENOUS, DOCUMENTARY, SPOR...",https://cdn.iview.abc.net.au/thumbs/i/rf/RF190...,[Director Laurence Billiet],"[abc1, australia, indigenous, docs, factual, f...",2021-06-30 07:05:00,G,3480.0,NaN,,Freeman,6
1073,1073,Family,Sleeping Beauty,NaN,Sleeping Beauty,The Australian Ballet presents Sleeping Beauty...,The Australian Ballet presents Sleeping Beauty...,"[ABC TV Plus, ARTS & CULTURE, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[],"[abc2, arts, entertainment, concert, ballet, c...",2021-06-18 07:00:00,G,7656.0,NaN,,Sleeping Beauty,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
838,838,Arts,Ochres,NaN,Ochres,"Told in five parts, Ochres was inspired by the...","Told in five parts, Ochres was inspired by the...","[ABC ARTS, ARTS & CULTURE, INDIGENOUS]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW229...,[],"[abcarts, arts, dance, bangarra, first-nations...",2021-06-18 07:00:00,G,3417.0,NaN,,Ochres,6
871,871,Arts,The Art of Collecting,NaN,The Art of Collecting,These short intimate stories by artist Kate Bl...,These short intimate stories by artist Kate Bl...,"[ABC TV Plus, ARTS & CULTURE]",https://cdn.iview.abc.net.au/thumbs/i/ac/AC194...,"[Cast David Capra, Sarah Goffman, Karla Dicken...","[abc2, australia, arts, culture, inspirational...",2021-06-16 22:31:40,G,1933.0,NaN,,The Art of Collecting,3
897,897,Arts,Whiteley,NaN,Whiteley,A visual journey into the private life and cre...,A visual journey into the private life and cre...,"[ABC TV, ABC TV Plus, ARTS & CULTURE, DOCUMENT...",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW138...,[Director James Bogle],"[abc1, abc2, abcarts, arts, docs, history, bio...",2021-06-01 07:00:00,M,5395.0,NaN,,Whiteley,3
209,209,Education,"Tell Your Story, Change Your World",NaN,"Tell Your Story, Change Your World","Only you can tell your own story, but getting ...","Only you can tell your own story, but getting ...","[ABC ME, EDUCATION]",https://cdn.iview.abc.net.au/thumbs/i/ck/CK189...,[],"[abc3, education, secondary-english]",2018-10-19 10:42:31,PG,875.0,NaN,,"Tell Your Story, Change Your World",6


In [116]:
#add to the top3 dataframe 2 random rows from suggestion connected
recommendations_expat_movie = top3.merge(suggestion.groupby('cluster').apply(lambda x: x.sample(2, replace=True)).reset_index(drop=True), on='cluster', how='left')
recommendations_expat_movie

,user_id,cluster,id,category,title,series,episode_name,description,description2,tags,image,directors_actors,tags2,publication_date,rating,duration_sec,season,episode,episode_title
0,1,1.0,932.0,Documentary,Women Of Steel,NaN,Women Of Steel,Outraged at sexual harassment in the work plac...,Outraged at sexual harassment in the work plac...,"[ABC TV Plus, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW292...,[],"[abc2, australia, docs, factual, history, inte...",2022-03-06 20:32:41,M,3325.0,NaN,,Women Of Steel
1,1,1.0,376.0,Documentary,Brazen Hussies,NaN,Brazen Hussies,A revealing documentary celebrating the legacy...,A revealing documentary celebrating the legacy...,"[ABC TV, DOCUMENTARY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO190...,[Director Catherine Dwyer],"[abc1, featured, docs, australia, factual, his...",2022-03-08 01:04:06,M,5318.0,NaN,,Brazen Hussies
2,1,12.0,424.0,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!
3,1,12.0,424.0,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!
4,1,5.0,376.0,Drama,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[Director John Clarke],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,NaN,,Murray Whelan: Stiff
5,1,5.0,376.0,Drama,Murray Whelan: Stiff,NaN,Murray Whelan: Stiff,"Murray Whelan, a young political advisers, mus...","Murray Whelan, a young political advisers, mus...","[ABC TV, DRAMA, MOVIES]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW281...,[Director John Clarke],"[abc1, australia, drama, crime, crime-drama, 2...",2021-06-25 07:00:00,M,5567.0,NaN,,Murray Whelan: Stiff
6,2,12.0,424.0,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!
7,2,12.0,424.0,Family,Stackorama!,NaN,Stackorama!,"Jaydyn Coggins, the best cup stacker in Austra...","Jaydyn Coggins, the best cup stacker in Austra...","[ABC TV Plus, DOCUMENTARY, SPORT, FAMILY]",https://cdn.iview.abc.net.au/thumbs/i/do/DO191...,[],"[abc2, docs, factual, sport, endearing, disabi...",2021-03-31 21:45:07,PG,3586.0,NaN,,Stackorama!
8,2,4.0,835.0,Drama,Children Of The Revolution,NaN,Children Of The Revolution,Few knew that Stalin spent his last night in t...,Few knew that Stalin spent his last night in t...,"[ABC TV, DRAMA, MOVIES, COMEDY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW303...,"[Cast Judy Davis, Richard Roxburgh, Sam Neill,...","[abc1, drama, film, comedy, australia, politic...",2021-10-01 07:00:00,M,6028.0,NaN,,Children Of The Revolution
9,2,4.0,835.0,Drama,Children Of The Revolution,NaN,Children Of The Revolution,Few knew that Stalin spent his last night in t...,Few knew that Stalin spent his last night in t...,"[ABC TV, DRAMA, MOVIES, COMEDY]",https://cdn.iview.abc.net.au/thumbs/i/zw/ZW303...,"[Cast Judy Davis, Richard Roxburgh, Sam Neill,...","[abc1, drama, film, comedy, australia, politic...",2021-10-01 07:00:00,M,6028.0,NaN,,Children Of The Revolution
